In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPool1D, UpSampling1D, Concatenate, Reshape, ZeroPadding1D, Input
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.optimizers import Adam,SGD,Nadam,RMSprop
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import random 
random.seed(0)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import model_from_json
import gc 
import time

In [2]:
HR_df = pd.read_csv ("newallscaledX.csv")

In [3]:
HR_df.head()

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,...,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,condition_no stress,condition_time pressure,HR
0,0.435965,0.296233,0.212099,0.475113,0.475064,0.123454,0.282572,0.097561,0.015735,0.277258,...,0.254282,0.008044,0.035395,0.475064,0.211924,0.947182,0.395743,1,0,69.499952
1,0.506031,0.379515,0.100855,0.352360,0.352353,0.069644,0.142132,0.000000,0.022531,0.222232,...,0.202013,0.041695,0.006793,0.352353,0.100622,0.966481,0.155015,0,0,64.363150
2,0.452786,0.343109,0.106784,0.510695,0.510700,0.048624,0.331641,0.036585,0.034048,0.165969,...,0.177376,0.011799,0.024316,0.510700,0.106298,0.944735,0.434949,0,0,67.450066
3,0.431575,0.331183,0.117737,0.482979,0.482965,0.059545,0.299492,0.024390,0.021064,0.196334,...,0.234906,0.006801,0.041645,0.482965,0.117344,0.968615,0.445258,1,0,68.809562
4,0.338444,0.258746,0.066260,0.648543,0.648539,0.011691,0.512690,0.036585,0.020405,0.222348,...,0.172403,0.003486,0.078401,0.648539,0.065139,0.992379,0.658937,1,0,74.565728


In [4]:
def save_models(model, filename):
  json_for_model = model.to_json()
  with open(filename, "w") as json_file_writer:
      json_file_writer.write(json_for_model)
  # serialize weights to HDF5
  model.save_weights(filename.split(".")[0] + ".h5")

def load_models(filename):
  json_for_model = open(filename, 'r')
  read_json = json_for_model.read()
  json_for_model.close()
  json_for_model = None
  model_loaded = model_from_json(read_json)
  # load weights into new model
  model_loaded.load_weights(filename.split(".")[0] + ".h5")
  gc.collect()
  return model_loaded

In [5]:
y = HR_df["HR"]
x = HR_df.drop(["HR"], axis = 1)
x = np.array(x)
y = np.array(y)

In [6]:
from sklearn.decomposition import PCA

#### Create PCA ####
pca = PCA(n_components=32)
x_pca = pca.fit_transform(x)

In [91]:
model = Sequential() #creating sequential model
model.add(Input(shape = (32,))) 
model.add(Dense(160, activation = "relu"))
model.add(Dense(1, activation = "relu"))

gc.collect()

30071

Train on 295431 samples, validate on 73858 samples
Epoch 1/7
295431/295431 [==============================] - 1s 4us/sample - loss: 46.4589 - mae: 46.4589 - val_loss: 5.6473 - val_mae: 5.6473
Epoch 2/7
295431/295431 [==============================] - 1s 3us/sample - loss: 1.7088 - mae: 1.7088 - val_loss: 0.7528 - val_mae: 0.7528
Epoch 3/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.5015 - mae: 0.5015 - val_loss: 0.3225 - val_mae: 0.3225
Epoch 4/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.2293 - mae: 0.2293 - val_loss: 0.1736 - val_mae: 0.1736
Epoch 5/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.1473 - mae: 0.1473 - val_loss: 0.1540 - val_mae: 0.1540
Epoch 6/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.1280 - mae: 0.1280 - val_loss: 0.1214 - val_mae: 0.1214
Epoch 7/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.1558 - mae: 0.1558 - val_loss:

295431/295431 [==============================] - 1s 3us/sample - loss: 0.0492 - mae: 0.0492 - val_loss: 0.0488 - val_mae: 0.0488
Epoch 6/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0463 - mae: 0.0463 - val_loss: 0.0507 - val_mae: 0.0507
Epoch 7/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0887 - mae: 0.0887 - val_loss: 0.1241 - val_mae: 0.1241
************************LOOP***************************** 14 LEARNING RATE 0.006332511891367892
Train on 295431 samples, validate on 73858 samples
Epoch 1/7
295431/295431 [==============================] - 1s 4us/sample - loss: 0.1373 - mae: 0.1373 - val_loss: 0.0624 - val_mae: 0.0624
Epoch 2/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0661 - mae: 0.0661 - val_loss: 0.0697 - val_mae: 0.0697
Epoch 3/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0748 - mae: 0.0748 - val_loss: 0.0754 - val_mae: 0.0754
Epoch 4/7
295431/295431 [====

295431/295431 [==============================] - 1s 4us/sample - loss: 0.0973 - mae: 0.0973 - val_loss: 0.0348 - val_mae: 0.0348
Epoch 2/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0295 - mae: 0.0295 - val_loss: 0.0309 - val_mae: 0.0309
Epoch 3/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0308 - mae: 0.0308 - val_loss: 0.0303 - val_mae: 0.0303
Epoch 4/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0325 - mae: 0.0325 - val_loss: 0.0344 - val_mae: 0.0344
Epoch 5/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0395 - mae: 0.0395 - val_loss: 0.0367 - val_mae: 0.0367
Epoch 6/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0625 - mae: 0.0625 - val_loss: 0.0601 - val_mae: 0.0601
Epoch 7/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0596 - mae: 0.0596 - val_loss: 0.0564 - val_mae: 0.0564
************************LOOP*********

295431/295431 [==============================] - 1s 4us/sample - loss: 0.0248 - mae: 0.0248 - val_loss: 0.0237 - val_mae: 0.0237
Epoch 6/7
295431/295431 [==============================] - 1s 4us/sample - loss: 0.0247 - mae: 0.0247 - val_loss: 0.0268 - val_mae: 0.0268
Epoch 7/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0252 - mae: 0.0252 - val_loss: 0.0308 - val_mae: 0.0308
************************LOOP***************************** 43 LEARNING RATE 0.0026179196186310546
Train on 295431 samples, validate on 73858 samples
Epoch 1/7
295431/295431 [==============================] - 1s 4us/sample - loss: 0.0593 - mae: 0.0593 - val_loss: 0.0277 - val_mae: 0.0277
Epoch 2/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0238 - mae: 0.0238 - val_loss: 0.0246 - val_mae: 0.0246
Epoch 3/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0236 - mae: 0.0236 - val_loss: 0.0272 - val_mae: 0.0272
Epoch 4/7
295431/295431 [===

295431/295431 [==============================] - 1s 4us/sample - loss: 0.0355 - mae: 0.0355 - val_loss: 0.0198 - val_mae: 0.0198
Epoch 2/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0208 - mae: 0.0208 - val_loss: 0.0235 - val_mae: 0.0235
Epoch 3/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0208 - mae: 0.0208 - val_loss: 0.0202 - val_mae: 0.0202
Epoch 4/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0209 - mae: 0.0209 - val_loss: 0.0228 - val_mae: 0.0228
Epoch 5/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0209 - mae: 0.0209 - val_loss: 0.0251 - val_mae: 0.0251
Epoch 6/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0211 - mae: 0.0211 - val_loss: 0.0278 - val_mae: 0.0278
Epoch 7/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0213 - mae: 0.0213 - val_loss: 0.0243 - val_mae: 0.0243
************************LOOP*********

...

295431/295431 [==============================] - 1s 5us/sample - loss: 0.0169 - mae: 0.0169 - val_loss: 0.0165 - val_mae: 0.0165
Epoch 2/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0163 - mae: 0.0163 - val_loss: 0.0166 - val_mae: 0.0166
Epoch 3/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0164 - mae: 0.0164 - val_loss: 0.0166 - val_mae: 0.0166
Epoch 4/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0164 - mae: 0.0164 - val_loss: 0.0165 - val_mae: 0.0165
Epoch 5/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0164 - mae: 0.0164 - val_loss: 0.0167 - val_mae: 0.0167
Epoch 6/7
295431/295431 [==============================] - 1s 4us/sample - loss: 0.0164 - mae: 0.0164 - val_loss: 0.0168 - val_mae: 0.0168
Epoch 7/7
295431/295431 [==============================] - 1s 4us/sample - loss: 0.0164 - mae: 0.0164 - val_loss: 0.0166 - val_mae: 0.0166
************************LOOP*********

295431/295431 [==============================] - 1s 4us/sample - loss: 0.0163 - mae: 0.0163 - val_loss: 0.0165 - val_mae: 0.0165
Epoch 6/7
295431/295431 [==============================] - 1s 4us/sample - loss: 0.0163 - mae: 0.0163 - val_loss: 0.0166 - val_mae: 0.0166
Epoch 7/7
295431/295431 [==============================] - 1s 4us/sample - loss: 0.0163 - mae: 0.0163 - val_loss: 0.0165 - val_mae: 0.0165
************************LOOP***************************** 159 LEARNING RATE 7.646760403106603e-05
Train on 295431 samples, validate on 73858 samples
Epoch 1/7
295431/295431 [==============================] - 1s 5us/sample - loss: 0.0165 - mae: 0.0165 - val_loss: 0.0165 - val_mae: 0.0165
Epoch 2/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0163 - mae: 0.0163 - val_loss: 0.0165 - val_mae: 0.0165
Epoch 3/7
295431/295431 [==============================] - 1s 4us/sample - loss: 0.0163 - mae: 0.0163 - val_loss: 0.0165 - val_mae: 0.0165
Epoch 4/7
295431/295431 [==

295431/295431 [==============================] - 1s 4us/sample - loss: 0.0163 - mae: 0.0163 - val_loss: 0.0164 - val_mae: 0.0164
Epoch 2/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0162 - mae: 0.0162 - val_loss: 0.0164 - val_mae: 0.0164
Epoch 3/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0162 - mae: 0.0162 - val_loss: 0.0165 - val_mae: 0.0165
Epoch 4/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0162 - mae: 0.0162 - val_loss: 0.0164 - val_mae: 0.0164
Epoch 5/7
295431/295431 [==============================] - 1s 4us/sample - loss: 0.0162 - mae: 0.0162 - val_loss: 0.0164 - val_mae: 0.0164
Epoch 6/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0162 - mae: 0.0162 - val_loss: 0.0164 - val_mae: 0.0164
Epoch 7/7
295431/295431 [==============================] - 1s 3us/sample - loss: 0.0162 - mae: 0.0162 - val_loss: 0.0164 - val_mae: 0.0164
************************LOOP*********

295431/295431 [==============================] - 1s 5us/sample - loss: 0.0161 - mae: 0.0161 - val_loss: 0.0164 - val_mae: 0.0164
Epoch 6/7
295431/295431 [==============================] - 1s 4us/sample - loss: 0.0161 - mae: 0.0161 - val_loss: 0.0164 - val_mae: 0.0164
Epoch 7/7
295431/295431 [==============================] - 1s 4us/sample - loss: 0.0162 - mae: 0.0162 - val_loss: 0.0164 - val_mae: 0.0164
************************LOOP***************************** 188 LEARNING RATE 3.161242240311002e-05
Train on 295431 samples, validate on 73858 samples
Epoch 1/7
295431/295431 [==============================] - 1s 5us/sample - loss: 0.0162 - mae: 0.0162 - val_loss: 0.0164 - val_mae: 0.0164
Epoch 2/7
295431/295431 [==============================] - 1s 4us/sample - loss: 0.0162 - mae: 0.0162 - val_loss: 0.0164 - val_mae: 0.0164
Epoch 3/7
295431/295431 [==============================] - 1s 4us/sample - loss: 0.0161 - mae: 0.0161 - val_loss: 0.0163 - val_mae: 0.0163
Epoch 4/7
295431/295431 [==

KeyboardInterrupt: 

In [7]:
from sklearn.ensemble import RandomForestRegressor

In [8]:
rf = RandomForestRegressor(
    n_estimators=10,
    criterion='mae',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=1,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x_pca,y, test_size = 0.2)

In [ ]:
rf.fit(x_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
rf.score(x_test, y_test)

In [94]:
save_models(model, "HR_regressor.json")

In [95]:
model = load_models("HR_regressor.json")
model.load_weights("HR_regressor.h5")
opt = Adam(learning_rate=lr)
model.compile(optimizer= opt, loss ="mean_absolute_error",metrics =["mae"],)

In [96]:
test_data = pd.read_csv("testallscaledX.csv")

In [97]:
test_data.head()

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,...,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,condition_no stress,condition_time pressure,HR
0,0.500161,0.371305,0.102287,0.297993,0.297997,0.082960,0.099831,0.024691,0.021479,0.216440,...,0.982636,0.008122,0.009197,0.017364,0.160364,0.024427,0.012337,0,1,NaN
1,0.348242,0.263284,0.052550,0.714174,0.714175,0.000666,0.624365,0.098765,0.032901,0.237069,...,0.915446,0.109840,0.140114,0.084554,0.138629,0.004767,0.061292,1,0,NaN
2,0.425362,0.331833,0.114516,0.395513,0.395517,0.072183,0.179357,0.098765,0.029850,0.175855,...,0.926371,0.040115,0.039913,0.073629,0.185956,0.005536,0.053197,1,0,NaN
3,0.634477,0.424026,0.347897,0.519243,0.519133,0.199367,0.274112,0.345679,0.019001,0.298355,...,0.960101,0.019485,0.008357,0.039899,0.464555,0.010534,0.028539,0,1,NaN
4,0.293322,0.230327,0.045243,0.225343,0.225344,0.045504,0.050761,0.012346,0.064254,0.242369,...,0.720794,0.092585,0.186102,0.279206,0.077307,0.001141,0.214966,0,0,NaN


In [98]:
test_data = test_data.drop(["HR"],axis = 1)

test_data_pca = pca.transform(test_data)
test_data_pca = np.array(test_data_pca)

In [99]:
y_predicted = model.predict(test_data_pca)

In [100]:
output_df = pd.DataFrame(data = y_predicted, columns = ["HR"])

In [101]:
output_df.to_csv("Output.csv")